## This notebook contains the code which I used in order to finetune "Helsinki-NLP/opus-mt-tc-big-en-ar" model on 'WIT3: Web Inventory of Transcribed and Translated Talks' dataset for English to Arabic Machine Translation

#### Install required packages

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [ ]:
!pip install transformers[torch] accelerate -U

In [ ]:
!pip install accelerate==0.21.0 transformers[torch] -U

In [ ]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [ ]:
!pip install peft

  Using cached peft-0.12.0-py3-none-any.whl (296 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import modules

In [ ]:
from datasets import load_dataset
import transformers
from transformers import AutoModel
from datasets import DatasetDict
from datasets import load_from_disk
import datasets
import random
import pandas as pd

from transformers import AutoTokenizer

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from nltk.translate.bleu_score import corpus_bleu
from IPython.display import display, HTML

import os
import sacrebleu

from peft import LoraConfig, get_peft_model
import torch
import sys





Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version

In [ ]:
print(transformers.__version__)

4.44.2


In [3]:
model_checkpoint = "Helsinki-NLP/opus-mt-tc-big-en-ar"

#### Loading the model

In [5]:


# Load the model using the same checkpoint
model = AutoModel.from_pretrained(model_checkpoint)

# Print the model architecture
print(model)


Some weights of MarianModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-tc-big-en-ar and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MarianModel(
  (shared): Embedding(61247, 1024, padding_idx=61246)
  (encoder): MarianEncoder(
    (embed_tokens): Embedding(61247, 1024, padding_idx=61246)
    (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
    (layers): ModuleList(
      (0-5): 6 x MarianEncoderLayer(
        (self_attn): MarianAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): ReLU()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
  )

#### Loading the dataset

In [ ]:
dataset1 = load_dataset("ted_talks_iwslt", language_pair=("en" , "ar"), year="2014")

In [ ]:
dataset2 = load_dataset("ted_talks_iwslt", language_pair=("en" , "ar"), year="2015")

In [ ]:
dataset3 = load_dataset("ted_talks_iwslt", language_pair=("en" , "ar"), year="2016")

#### Merging the three dataset into one

In [ ]:
from datasets import concatenate_datasets
train_datasets = [dataset1['train'], dataset2['train'], dataset3['train']]
merged_train_dataset = concatenate_datasets(train_datasets)

#### Splitting dataset into train,eval and test sets

In [ ]:
# Split the merged dataset into train, eval, and test sets
# You can adjust the split sizes as needed
split_dataset = merged_train_dataset.train_test_split(test_size=0.2)  # Split into train and temp test
temp_dataset = split_dataset['test'].train_test_split(test_size=0.5)  # Further split temp test into eval and test

# Create final dataset dictionary with train, eval, and test sets
final_dataset = DatasetDict({
    'train': split_dataset['train'],
    'eval': temp_dataset['train'],
    'test': temp_dataset['test']
})


In [17]:
# # Optionally, save the dataset to disk
# final_dataset.save_to_disk('final_dataset')

In [18]:
# !zip -r /content/final_dataset.zip /content/final_dataset

In [19]:
# final_dataset

In [20]:

merged_train_dataset = load_from_disk(r'final_dataset\content\final_dataset')
merged_train_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 8585
    })
    eval: Dataset({
        features: ['translation'],
        num_rows: 1073
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1074
    })
})

In [21]:
final_dataset = merged_train_dataset

#### To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [27]:


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(final_dataset["train"])

,translation
0,"{'ar': 'سوكي كيم: هكذا يبدو الحال حينما تكون متخفيا في كوريا الشمالية', 'en': 'Suki Kim: This is what it's like to go undercover in North Korea'}"
1,"{'ar': 'ريجي واتس: يضللكم ريجي واتس بأكثر الطرق تسلية', 'en': 'Reggie Watts: Beats that defy boxes'}"
2,"{'ar': 'تشيب كونلي: قياس ما الذي يجعل الحياة ذات قيمة', 'en': 'Chip Conley: Measuring what makes life worthwhile'}"
3,"{'ar': 'إكتشف تيري مور بأنه كانه يقوم بربط حذائه بطريقة خاطئة. لهذا صعد الى مسرح تيد ليتشارك طريقة ربط أفضل على نسق تيد. (إشارة تاريخية: كان هذا أول حديث من جمهور تيد لثلاث دقائق عام 2005.)', 'en': 'Terry Moore found out he'd been tying his shoes the wrong way his whole life. In the spirit of he takes the stage to share a better way. (Historical note: This was the very first 3-minute audience talk given from the TED stage, in 2005.)'}"
4,"{'ar': 'عالم الأحياء ريتشارد دوكينز يشرح مسئلة ""التفكير في الغير محتمل "" بالنظر في كيف يحد الإطار المرجعي للإنسان من فهمنا للكون.', 'en': 'Biologist Richard Dawkins makes a case for ""thinking the improbable"" by looking at how the human frame of reference limits our understanding of the universe.'}"


# Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

If you downloaded the model manually, you can provide model present directory instead of model_checkpoint

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

c:\Users\azam\Desktop\Azam_MT_Finetuning\env\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\azam\Desktop\Azam_MT_Finetuning\env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


You can directly call this tokenizer on one sentence or a pair of sentences:

In [30]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[14903, 7, 29396, 21267, 26543, 13, 25897], [29398, 16742, 3959, 26543, 22, 25897]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the as_target_tokenizer context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [31]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[14504, 11237, 7, 137, 29238, 16741, 137, 21266, 25899, 11598, 11407, 13, 25897], [29240, 16741, 16742, 137, 3788, 21588, 25899, 11598, 11407, 22, 25897]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


c:\Users\azam\Desktop\Azam_MT_Finetuning\env\lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


We can then write the function that will preprocess our samples. We just feed them to the tokenizer with the argument truncation=True. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [32]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ar"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [33]:
final_dataset['train'][:2]

{'translation': [{'ar': 'روبن تشيس: الفكرة خلف Zipcar (والفكرة المقبلة)',
   'en': 'Robin Chase: The idea behind Zipcar (and what comes next)'},
  {'ar': 'حول العالم، النساء سيحاربن للحصول على المساواة في أبسط الأشياء الأساسية مثل الحق في قيادة السيارة والزواج عندما تقرر هي ذلك. ولكن كيف يتم ذكر ذكر الجميع، نساء ورجالا كحلفاء من اجل التغيير.؟ التقوا باليزابيث نيامايارو مخترعة حملة #هومن أجلهاهي على تويتر والتي حققت 1.2 بليون محادثة حول المساواة في العالم. ولقد دعتنا جميعنا لنشارك كحلفاء ذات إنسانية مشتركة.',
   'en': 'Around the world, women still struggle for equality in basic matters like access to education, equal pay and the right to vote. But how to enlist everyone, men and women, as allies for change? Meet Elizabeth Nyamayaro, head of UN Women’s HeForShe initiative, which has created more than 2.4 billion social media conversations about a more equal world. She invites us all to join in as allies in our shared humanity.'}]}

In [34]:
preprocess_function(final_dataset['train'][:2])

{'input_ids': [[25632, 6989, 11, 29275, 15578, 5295, 32491, 6498, 164, 3847, 31808, 7687, 20585, 71, 25897], [4345, 29273, 32125, 7, 32062, 28037, 28242, 13221, 11705, 15916, 5133, 19079, 18143, 2839, 29600, 11030, 7, 11703, 22139, 3848, 29273, 25513, 29600, 31457, 22, 6256, 15310, 29600, 11560, 12017, 7, 19301, 3848, 32062, 7, 4422, 3606, 13221, 6920, 18, 19243, 11226, 20243, 32295, 32279, 4339, 7, 14801, 21082, 30419, 32064, 40, 25896, 14797, 13222, 26790, 16230, 7, 31847, 14733, 8816, 19943, 29250, 1422, 5493, 27356, 19209, 8478, 2752, 2670, 19943, 11703, 32125, 22, 26791, 16648, 30899, 3572, 29600, 17081, 15916, 4422, 3606, 15916, 21623, 26756, 15385, 22, 25897]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command

In [35]:
tokenized_datasets = final_dataset.map(preprocess_function, batched=True)

In [37]:
# pip install torch torchvision torchaudio

#### Check for GPU

In [ ]:


import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU.


In [37]:
# !nvidia-smi


In [38]:
# !pip uninstall torch torchvision torchaudio


In [39]:
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu12x


In [40]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


CUDA available: True
CUDA version: 11.8
Number of GPUs: 1
Device name: Tesla T4


In [31]:
tokenized_datasets['train'].features['translation']

Translation(languages=['en', 'ar'], id=None)

In [32]:
# # Just evaluating by azam

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
# bleu_metric = load_metric("bleu")

# # Reduce the dataset size for evaluation
# small_dataset = final_dataset["test"].select(range(100))  # Select the first 100 examples

# def generate_translations(batch):
#     inputs = [example["en"] for example in batch["translation"]]
#     inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).input_ids
#     outputs = model.generate(inputs)
#     batch["predicted_translation"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return batch

# # Apply the function to generate translations on the smaller dataset
# results = small_dataset.map(generate_translations, batched=True, batch_size=10)

# # Compute BLEU score
# def compute_bleu(predictions, references):
#     pred_tokens = [pred.split() for pred in predictions]
#     ref_tokens = [[ref.split()] for ref in references]
#     bleu = bleu_metric.compute(predictions=pred_tokens, references=ref_tokens)
#     return bleu["bleu"]

# # Extract predictions and references
# predictions = results["predicted_translation"]
# references = [example["ar"] for example in results["translation"]]

# # Calculate BLEU score
# bleu_score = compute_bleu(predictions, references)
# print(f"BLEU score: {bleu_score}")

#### Baseline model's performance on WIT3 dataset

In [ ]:
bleu_metric = corpus_bleu

# Reduce the dataset size for evaluation
small_dataset = final_dataset["test"]

def generate_translations(batch):
    inputs = [example["en"] for example in batch["translation"]]
    inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).input_ids
    outputs = model.generate(inputs)
    batch["predicted_translation"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

# Apply the function to generate translations on the smaller dataset
results = small_dataset.map(generate_translations, batched=True, batch_size=10)

In [28]:
# Compute BLEU score
def compute_bleu(predictions, references):
    pred_tokens = [pred.split() for pred in predictions]
    ref_tokens = [[ref.split()] for ref in references]
    bleu = bleu_metric(ref_tokens,pred_tokens)
    return bleu

In [ ]:
# Extract predictions and references
predictions = results["predicted_translation"]
references = [example["ar"] for example in results["translation"]]


In [ ]:
# Calculate BLEU score
bleu_score = compute_bleu(predictions, references)
print(f"BLEU score: {bleu_score}")

BLEU score: 0.11230733833443755


## To test how model makes prediction

In [ ]:
## Test translation

def translate_en_to_ar(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

    # Generate the translated text
    translated = model.generate(**inputs)

    # Decode the generated text to get the Arabic sentence
    translated_sentence = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_sentence

# Example sentence in English
english_sentence = "I love learning new things about machine learning."

# Translate the sentence to Arabic
arabic_translation = translate_en_to_ar(english_sentence)
print(f"Original: {english_sentence}")
print(f"Translated: {arabic_translation}")

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [41]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

print(model)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61247, 1024, padding_idx=61246)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61247, 1024, padding_idx=61246)
      (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

To instantiate a Seq2SeqTrainer, we will need to define three more things. The most important is the [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [42]:

print(sys.executable)


c:\Users\azam\Desktop\Azam_MT_Finetuning\env\Scripts\python.exe


#### Preparing model for LoRA Finetuning

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install torch transformers accelerate -U

In [ ]:
import accelerate
print(accelerate.__version__)


In [ ]:
# LoRA code


# Freeze the embedding layers and specific encoder/decoder layers
def freeze_layers(model):
    # Freeze embedding layers and positional embeddings
    model.model.shared.requires_grad_(False)
    model.model.encoder.embed_tokens.requires_grad_(False)
    model.model.encoder.embed_positions.requires_grad_(False)
    model.model.decoder.embed_tokens.requires_grad_(False)
    model.model.decoder.embed_positions.requires_grad_(False)
    
    # Freeze encoder layers 0-3
    for i in range(3):  # Layers 0, 1, 2
        model.model.encoder.layers[i].requires_grad_(False)
    
    # Freeze decoder layers 0-3
    for i in range(3):  # Layers 0, 1, 2
        model.model.decoder.layers[i].requires_grad_(False)

# Apply the freezing function
freeze_layers(model)
print(model)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61247, 1024, padding_idx=61246)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61247, 1024, padding_idx=61246)
      (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

In [ ]:
# LoRa code check

# Check which layers are frozen and which are trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer {name} is trainable.")
    else:
        print(f"Layer {name} is frozen.")

Layer model.shared.weight is frozen.
Layer model.encoder.embed_positions.weight is frozen.
Layer model.encoder.layers.0.self_attn.k_proj.weight is frozen.
Layer model.encoder.layers.0.self_attn.k_proj.bias is frozen.
Layer model.encoder.layers.0.self_attn.v_proj.weight is frozen.
Layer model.encoder.layers.0.self_attn.v_proj.bias is frozen.
Layer model.encoder.layers.0.self_attn.q_proj.weight is frozen.
Layer model.encoder.layers.0.self_attn.q_proj.bias is frozen.
Layer model.encoder.layers.0.self_attn.out_proj.weight is frozen.
Layer model.encoder.layers.0.self_attn.out_proj.bias is frozen.
Layer model.encoder.layers.0.self_attn_layer_norm.weight is frozen.
Layer model.encoder.layers.0.self_attn_layer_norm.bias is frozen.
Layer model.encoder.layers.0.fc1.weight is frozen.
Layer model.encoder.layers.0.fc1.bias is frozen.
Layer model.encoder.layers.0.fc2.weight is frozen.
Layer model.encoder.layers.0.fc2.bias is frozen.
Layer model.encoder.layers.0.final_layer_norm.weight is frozen.
Lay

In [ ]:
# LoRA code 2

# from peft import  prepare_model_for_int8_training

# Apply LoRA only to higher encoder/decoder layers 4-5 (attention and feedforward)
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],  # Apply LoRA to these modules
    lora_dropout=0.1,  # Regularization
    bias="none"  # LoRA applied only to weights
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

In [48]:
# 5. Ensure LoRA is applied only to the remaining layers
for name, param in lora_model.named_parameters():
    # Only apply LoRA to layers 3 and onward
    if "encoder.layers.0" in name or "encoder.layers.1" in name or "encoder.layers.2" in name:
        param.requires_grad = False  # Ensure these layers remain frozen
    if "decoder.layers.0" in name or "decoder.layers.1" in name or "decoder.layers.2" in name:
        param.requires_grad = False  # Ensure these layers remain frozen

# Print the status of each parameter (trainable or frozen)
for name, param in lora_model.named_parameters():
    if param.requires_grad:
        print(f"Trainable Layer: {name}")
    else:
        print(f"Frozen Layer: {name}")

Frozen Layer: base_model.model.model.shared.weight
Frozen Layer: base_model.model.model.encoder.embed_positions.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.bias
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.bias
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.q_proj.base_layer.weight
Frozen Layer: base_model.model.model.encoder.layers.0.self_attn.q_pr

In [49]:
print(lora_model)

PeftModel(
  (base_model): LoraModel(
    (model): MarianMTModel(
      (model): MarianModel(
        (shared): Embedding(61247, 1024, padding_idx=61246)
        (encoder): MarianEncoder(
          (embed_tokens): Embedding(61247, 1024, padding_idx=61246)
          (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
          (layers): ModuleList(
            (0-5): 6 x MarianEncoderLayer(
              (self_attn): MarianAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=1024, bias=False)
                  )
     

In [ ]:
# LoRa code check

# Check which layers are frozen and which are trainable
for name, param in lora_model.named_parameters():
    if param.requires_grad:
        print(f"Layer {name} is trainable.")
    else:
        print(f"Layer {name} is frozen.")

Layer base_model.model.model.shared.weight is frozen.
Layer base_model.model.model.encoder.embed_positions.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.bias is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.bias is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.q_proj.base_layer.weight is frozen.
Layer base_model.model.model.encode

In [52]:
model = lora_model

In [55]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True
)

c:\Users\azam\Desktop\Azam_MT_Finetuning\env\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the cell and customize the weight decay. Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the predict_with_generate option (to properly generate translation) and activate mixed precision training (to go a bit faster).

Model will save under **{model_name}-finetuned-{source_lang}-to-{target_lang}** directory

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [57]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [59]:
import numpy as np

def postprocess_text(preds, labels):
    """
    Post-processes the predictions and labels by stripping extra spaces.
    """
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]  # Flatten labels since you handle nesting later
    return preds, labels


def compute_bleu(predictions, references):
    """
    Computes BLEU score for the predictions and references.
    Assumes that the predictions and references are already tokenized as text strings.
    """
    # Ensure the predictions and references are lists of strings
    if not all(isinstance(pred, str) for pred in predictions):
        raise TypeError("Each element of `predictions` should be a string.")
    
    if not all(isinstance(ref, list) and isinstance(ref[0], str) for ref in references):
        raise TypeError("Each element of `references` should be a list of strings.")

    # Compute BLEU score using sacrebleu, which expects raw text (not tokenized)
    bleu = sacrebleu.corpus_bleu(predictions, references)

    return bleu.score

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in labels (used for padding in some models) with the tokenizer's pad token ID
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process predictions and labels
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Convert decoded labels to a list of lists (BLEU requires each reference to be a list of one reference sentence)
    decoded_labels = [[label] for label in decoded_labels]  # Each label is a list of one reference sentence

    # Compute BLEU score
    bleu_score = compute_bleu(predictions=decoded_preds, references=decoded_labels)
    
    try:
        print(fr'real = {decoded_labels[0]}\n pred = {decoded_preds[0]}')
    except Exception as e:
        print('Exception:',e)
        print(fr'real = {decoded_labels}\n pred = {decoded_preds}')

    # Prepare the result dictionary
    result = {"bleu": bleu_score}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    # Round all results to 4 decimal places
    result = {k: round(v, 4) for k, v in result.items()}
    
    return result



The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [60]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [61]:
print(model)

PeftModel(
  (base_model): LoraModel(
    (model): MarianMTModel(
      (model): MarianModel(
        (shared): Embedding(61247, 1024, padding_idx=61246)
        (encoder): MarianEncoder(
          (embed_tokens): Embedding(61247, 1024, padding_idx=61246)
          (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
          (layers): ModuleList(
            (0-5): 6 x MarianEncoderLayer(
              (self_attn): MarianAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=1024, bias=False)
                  )
     

In [62]:
# Azam LoRa code check

# Check which layers are frozen and which are trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer {name} is trainable.")
    else:
        print(f"Layer {name} is frozen.")

Layer base_model.model.model.shared.weight is frozen.
Layer base_model.model.model.encoder.embed_positions.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.base_layer.bias is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.base_layer.bias is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight is frozen.
Layer base_model.model.model.encoder.layers.0.self_attn.q_proj.base_layer.weight is frozen.
Layer base_model.model.model.encode

We can now finetune our model by just calling the train method:

In [63]:
trainer.train()

  3%|▎         | 500/16110 [03:36<1:59:57,  2.17it/s]

{'loss': 1.9918, 'grad_norm': 0.7758260369300842, 'learning_rate': 1.937926753569212e-05, 'epoch': 0.93}


                                                     
  3%|▎         | 537/16110 [07:07<1:35:42,  2.71it/s]

real = ['يعتمد مروجي الإشاعات على فكرة أن الروبوتات ببساطة ستحل محل الناس في الوظائف وفي الحقيقة قد يصبحوا معاونينا الأساسيين الذين يحررونا من قضاء الوقت في التحديات البسيطة والميكانيكية ويشير رودني بروكس إلى مدى قيمة ذلك حيث أن عدد البالغين الذي هم في سن العمل في انخفاض بينما عدد المتقاعدين في ارتفاع وقد قدمنا إلى باكستر الروبوت ذو العينان اللتان تتحركان والذراع التي تستجيب للمسة اليد والذي قد يعمل جنباً إلى جنب المسنين ويتعلم مساعدتهم في المنزل أيضاً.']\n pred = يلعب المخيفون على فكرة أن الروبوتات ستحل ببساطة محل الأشخاص في العمل. في الواقع ، يمكن أن يصبحوا متعاونين أساسيين لدينا ، مما يحررنا لقضاء بعض الوقت في تحديات أقل دنيوية وميكانيكية. يشير رودني بروكس إلى مدى أهمية ذلك مع انخفاض عدد البالغين في سن العمل وعدد المتقاعدين. يقدم لنا باكستر ، الروبوت بعيون تتحرك وأذرع تتفاعل مع اللمس ، والتي يمكن أن تساعد على حد سواء.
{'eval_loss': 1.8500359058380127, 'eval_bleu': 18.3316, 'eval_gen_len': 36.5107, 'eval_runtime': 194.3248, 'eval_samples_per_second': 5.522, 'eval_steps_per_second': 0

  6%|▌         | 1000/16110 [10:30<1:53:34,  2.22it/s] 

...

 99%|█████████▉| 16000/16110 [3:31:56<00:50,  2.18it/s]  

{'loss': 1.713, 'grad_norm': 2.298205614089966, 'learning_rate': 1.3656114214773434e-07, 'epoch': 29.8}


                                                       
100%|██████████| 16110/16110 [3:35:57<00:00,  1.24it/s]

real = ['يعتمد مروجي الإشاعات على فكرة أن الروبوتات ببساطة ستحل محل الناس في الوظائف وفي الحقيقة قد يصبحوا معاونينا الأساسيين الذين يحررونا من قضاء الوقت في التحديات البسيطة والميكانيكية ويشير رودني بروكس إلى مدى قيمة ذلك حيث أن عدد البالغين الذي هم في سن العمل في انخفاض بينما عدد المتقاعدين في ارتفاع وقد قدمنا إلى باكستر الروبوت ذو العينان اللتان تتحركان والذراع التي تستجيب للمسة اليد والذي قد يعمل جنباً إلى جنب المسنين ويتعلم مساعدتهم في المنزل أيضاً.']\n pred = يلعب المشجعون على فكرة أن الروبوتات سوف تحل ببساطة محل الناس في العمل. في الواقع، يمكن أن تصبح المتعاونين الأساسيين لدينا، مما يتيح لنا قضاء بعض الوقت على تحديات أقل دنيوية وميكانيكية. رودني بروكس يشير إلى مدى قيمة هذا يمكن أن يكون مع انخفاض عدد البالغين في سن العمل وعدد المتقاعدين يتضخم. انه يقدم لنا باكستر، الروبوت مع العيون التي تتحرك والأذرع التي تتفاعل مع اللمس، والتي يمكن أن تعمل جنبا إلى جنب مع مساعدة السكان المسنين.
{'eval_loss': 1.704479694366455, 'eval_bleu': 15.052, 'eval_gen_len': 36.0755, 'eval_runtime': 192.6309

TrainOutput(global_step=16110, training_loss=1.7857395487050696, metrics={'train_runtime': 12957.6878, 'train_samples_per_second': 19.876, 'train_steps_per_second': 1.243, 'total_flos': 2.790602752602931e+16, 'train_loss': 1.7857395487050696, 'epoch': 30.0})

In [64]:
trainer.save_model()

In [65]:
# # Resume training (if needed)
# trainer.train(resume_from_checkpoint=True)

In [66]:

for dirname, _, filenames in os.walk('opus-mt-tc-big-en-ar-finetuned-en-to-ar'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

opus-mt-tc-big-en-ar-finetuned-en-to-ar\adapter_config.json
opus-mt-tc-big-en-ar-finetuned-en-to-ar\adapter_model.safetensors
opus-mt-tc-big-en-ar-finetuned-en-to-ar\README.md
opus-mt-tc-big-en-ar-finetuned-en-to-ar\source.spm
opus-mt-tc-big-en-ar-finetuned-en-to-ar\special_tokens_map.json
opus-mt-tc-big-en-ar-finetuned-en-to-ar\target.spm
opus-mt-tc-big-en-ar-finetuned-en-to-ar\tokenizer_config.json
opus-mt-tc-big-en-ar-finetuned-en-to-ar\training_args.bin
opus-mt-tc-big-en-ar-finetuned-en-to-ar\vocab.json
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\adapter_config.json
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\adapter_model.safetensors
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\optimizer.pt
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\README.md
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\rng_state.pth
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoint-15500\scheduler.pt
opus-mt-tc-big-en-ar-finetuned-en-to-ar\checkpoin

Load the model and translate some text from English to Arabic

In [71]:
## Test translation

def translate_en_to_ar(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

    # Generate the translated text
    translated = model.generate(**inputs)

    # Decode the generated text to get the Arabic sentence
    translated_sentence = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_sentence

# Example sentence in English
english_sentence = "I love learning new things about machine learning."

# Translate the sentence to Arabic
arabic_translation = translate_en_to_ar(english_sentence)
print(f"Original: {english_sentence}")
print(f"Translated: {arabic_translation}")

Original: I love learning new things about machine learning.
Translated: أحب تعلم أشياء جديدة عن التعلم الآلي.


In [72]:
# Just evaluating by azam

bleu_metric = corpus_bleu

# Reduce the dataset size for evaluation
small_dataset = final_dataset["test"]

def generate_translations(batch):
    inputs = [example["en"] for example in batch["translation"]]
    inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True).input_ids
    outputs = model.generate(inputs)
    batch["predicted_translation"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

# Apply the function to generate translations on the smaller dataset
results = small_dataset.map(generate_translations, batched=True, batch_size=10)

Map: 100%|██████████| 1074/1074 [33:10<00:00,  1.85s/ examples]


In [73]:
# Compute BLEU score
def compute_bleu(predictions, references):
    pred_tokens = [pred.split() for pred in predictions]
    ref_tokens = [[ref.split()] for ref in references]
    bleu = bleu_metric(ref_tokens,pred_tokens)
    return bleu

In [74]:


# Extract predictions and references
predictions = results["predicted_translation"]
references = [example["ar"] for example in results["translation"]]


#### Finetuned model's Performance

In [ ]:
# Calculate BLEU score
bleu_score = compute_bleu(predictions, references)
print(f"BLEU score: {bleu_score}")

BLEU score: 0.17426418957014538


In [ ]:
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

loading configuration file opus-mt-en-ro-finetuned-en-to-ro/checkpoint-38000/config.json
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-en-ro",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      59542
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 59542,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    

['Numele meu este Sarah şi locuiesc la Londra']